In [1]:
from poppy.creatures import PoppyHumanoid
import math
import time
import numpy as np

In [2]:
poppy = PoppyHumanoid()

In [3]:
def get_Jacobian(theta):
    ARM_LENGTH_1 = 15.1
    ARM_LENGTH_2 = 10.1
    return np.array([
                [-np.sin(theta[0])*ARM_LENGTH_1 - ARM_LENGTH_2*np.sin(theta[0]+theta[1]), 
                -ARM_LENGTH_2*np.sin(theta[0]+theta[1])], 

                [np.cos(theta[0])*ARM_LENGTH_1 + ARM_LENGTH_2*np.cos(theta[0]+theta[1]), 
                ARM_LENGTH_2*np.cos(theta[0]+theta[1])]
            ])

def get_control(distance, theta):
    J = get_Jacobian(theta)
    u = np.dot(np.linalg.pinv(J), distance)
    return u

def get_left_end_effector_position(theta):
    ARM_LENGTH_1 = 15.1
    ARM_LENGTH_2 = 10.1
    OFFSET = 0.0 #10.95
    pos = np.array([0.0, 0.0])
    pos[0] = ARM_LENGTH_1*np.cos(theta[0]) + ARM_LENGTH_2*np.cos(theta[0]+theta[1])
    pos[1] = OFFSET + ARM_LENGTH_1*np.sin(theta[0]) + ARM_LENGTH_2*np.sin(theta[0]+theta[1])
    return pos

def set_action(action):
    if action==1:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position - 1, 0, wait=True)
    elif action==2:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position + 1, 0, wait=True) 
    elif action==3:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position - 1, 0, wait=True)
    elif action==4:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position + 1, 0, wait=True)
    return

In [7]:
l_motors = [poppy.l_shoulder_y, poppy.l_shoulder_x, poppy.l_arm_z, poppy.l_elbow_y]
r_motors = [poppy.r_shoulder_y, poppy.r_shoulder_x, poppy.r_arm_z, poppy.r_elbow_y]
for m in l_motors+r_motors:
    m.compliant = False

# left init
poppy.l_shoulder_y.goto_position(-90,1,wait=True)
poppy.l_arm_z.goto_position(-90,1,wait=True)

# right init
poppy.r_shoulder_y.goto_position(-90,1,wait=True)
poppy.r_arm_z.goto_position(90,1,wait=True)

# some starting position
poppy.l_elbow_y.goto_position(-90,1,wait=True)
poppy.l_shoulder_x.goto_position(60, 1, wait=True)

theta = [poppy.l_shoulder_x.present_position*np.pi/180.0, poppy.l_elbow_y.present_position*np.pi/180.0]
hand = get_left_end_effector_position(theta)
goal = np.array([25., 0.])

print hand, goal

[ 16.41144214   7.88849828] [ 25.   0.]


In [8]:
for i in range(300):
    theta = [poppy.l_shoulder_x.present_position*np.pi/180.0, poppy.l_elbow_y.present_position*np.pi/180.0]
    hand = get_left_end_effector_position(theta)
    distance = goal - hand
    print i, theta, goal, hand, np.linalg.norm(distance)
    
    if np.linalg.norm(distance) < 1.0:
        break
    
    u = get_control(distance, theta)
    
    waiting = False
    if np.argmax(np.abs(u)) == 0:
        if u[0] < 0:
            poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position - 1, 0, wait=waiting)
        else:
            poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position + 1, 0, wait=waiting) 
    else:
        if u[1] < 0:
            poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position - 1, 0, wait=waiting)
        else:
            poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position + 1, 0, wait=waiting)
    
    time.sleep(0.1)

0 [1.036027443983834, -1.5643386085625173] [ 25.   0.] [ 16.41855925   7.90067287] 11.6645513049
1 [1.0192722831646883, -1.5657348719641129] [ 25.   0.] [ 16.54130543   7.61242753] 11.3797436996
2 [1.0023425894203433, -1.5657348719641129] [ 25.   0.] [ 16.66780491   7.33131078] 11.0983599137
3 [0.9854128956759985, -1.5657348719641129] [ 25.   0.] [ 16.78952727   7.04809282] 10.8206966008
4 [0.9700539982584483, -1.5657348719641129] [ 25.   0.] [ 16.8957937    6.78940304] 10.572329615
5 [0.9546951008408984, -1.5657348719641129] [ 25.   0.] [ 16.99807456   6.5291117 ] 10.3276381817
6 [0.940906999750143, -1.5657348719641129] [ 25.   0.] [ 17.08648002   6.29412734] 10.1113716912
7 [0.9241518389309975, -1.5657348719641129] [ 25.   0.] [ 17.18953587   6.00697054] 9.85327585613
8 [0.9087929415134475, -1.5657348719641129] [ 25.   0.] [ 17.27976525   5.74226011] 9.62162022233
9 [0.9072221451866523, -1.5535175672001527] [ 25.   0.] [ 17.36367617   5.81316221] 9.59720253217
10 [0.9056513488598577,

In [9]:
poppy.close()